In [1]:
config = {
     'bootstrap.servers': 'localhost:19092'     
     #'security.protocol': 'SASL_SSL',
     #'sasl.mechanisms': 'PLAIN',
     #'sasl.username': '<CLUSTER_API_KEY>', 
     #'sasl.password': '<CLUSTER_API_SECRET>'
     }

In [2]:
from confluent_kafka.admin import (AdminClient, NewTopic, 
                                   ConfigResource)

In [3]:
kadmin = AdminClient(config)

In [4]:
def topic_exists(admin, topic):
    metadata = admin.list_topics()
    for t in iter(metadata.topics.values()):
        print("Topic {} is available".format(t.topic))
        if t.topic == topic:
            return True
    return False

In [5]:
DAILY_DEMAND = "DAILY_DEMAND"

In [6]:
def create_topic(admin, topic):
    new_topic = NewTopic(topic, num_partitions=6, replication_factor=1) 
    result_dict = admin.create_topics([new_topic])
    for topic, future in result_dict.items():
        try:
            future.result()  # The result itself is None
            print("Topic {} created".format(topic))
        except Exception as e:
            print("Failed to create topic {}: {}".format(topic, e))

In [7]:
def get_max_size(admin, topic):
    resource = ConfigResource('topic', topic)
    result_dict = admin.describe_configs([resource])
    config_entries = result_dict[resource].result()
    max_size = config_entries['max.message.bytes']
    return max_size.value

In [8]:
# set max.message.bytes for topic
def set_max_size(admin, topic, max_k):
    config_dict = {'max.message.bytes': str(max_k*1024)}
    resource = ConfigResource('topic', topic, config_dict)
    result_dict = admin.alter_configs([resource])
    result_dict[resource].result()

In [9]:
# Create Admin client
admin = AdminClient(config)
topic_name = DAILY_DEMAND
max_msg_k = 50

# Create topic if it doesn't exist
if not topic_exists(admin, topic_name):
    create_topic(admin, topic_name)

# Check max.message.bytes config and set if needed
current_max = get_max_size(admin, topic_name)
if current_max != str(max_msg_k * 1024):
    print(f'Topic, {topic_name} max.message.bytes is {current_max}.')
    set_max_size(admin, topic_name, max_msg_k)

# Verify config was set 
new_max = get_max_size(admin, topic_name)
print(f'Now max.message.bytes for topic {topic_name} is {new_max}')

Topic _schemas is available
Topic __consumer_offsets is available
Topic DAILY_DEMAND created
Topic, DAILY_DEMAND max.message.bytes is 1048588.
Now max.message.bytes for topic DAILY_DEMAND is 51200


/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_21671/471056771.py:5: DeprecationWarning: alter_configs has been deprecated. Use incremental_alter_configs instead.
  result_dict = admin.alter_configs([resource])
